In [4]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import csv
import os
from concurrent.futures import ThreadPoolExecutor

# Load the data from the CSV file
data = []
with open('metadata.csv', 'r') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    for row in csv_reader:
        data.append(row)
        
output_folder = "waveform_data"
os.makedirs(output_folder, exist_ok=True)

def process_row(args):
    row, i = args
    output_file = os.path.join(output_folder, f"output_file{i}.mseed")
    
    # Check if the file already exists
    if os.path.exists(output_file):
        return f"Skipped row {i}: File already exists"
    
    try:
        start_time = UTCDateTime(row['trace_noise_start_time'])
        end_time = UTCDateTime(row['trace_end_time'])
        
        if start_time > end_time:
            return f"Skipped row {i}: start_time > end_time"

        client = Client("IRIS")
        network = "XB"
        station = "ELYSE"
        location = "02"
        channel = "BH*"
        waveforms = client.get_waveforms(starttime=start_time,
                                        endtime=end_time,
                                        station=station,
                                        channel=channel,
                                        location=location,
                                        network=network)
        waveforms.write(output_file, format="MSEED")
        return f"Successfully processed row {i}"
    except Exception as e:
        return f"Error processing row {i}: {e}"

with ThreadPoolExecutor(max_workers=10) as executor:
    results = executor.map(process_row, [(row, i) for i, row in enumerate(data)])
    
    for result in results:
        print(result)       

Successfully processed row 0
Error processing row 1: No data available for request.
Detailed response of server:


Error processing row 2: No data available for request.
Detailed response of server:


Error processing row 3: No data available for request.
Detailed response of server:


Successfully processed row 4
Error processing row 5: No data available for request.
Detailed response of server:


Successfully processed row 6
Error processing row 7: No data available for request.
Detailed response of server:


Successfully processed row 8
Successfully processed row 9
Error processing row 10: No data available for request.
Detailed response of server:


Error processing row 11: No data available for request.
Detailed response of server:


Successfully processed row 12
Successfully processed row 13
Error processing row 14: No data available for request.
Detailed response of server:


Successfully processed row 15
Error processing row 16: No data available for request.
Detailed response 

In [5]:
import os
import shutil
import subprocess
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime

def log_message(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"[{timestamp}] {message}"

def run_seisglitch_commands(directory):
    command = "seisglitch detect config.yml && seisglitch remove config.yml"
    try:
        subprocess.run(command, check=True, cwd=directory, shell=True)
        return log_message(f"Successfully ran commands in {directory}")
    except subprocess.CalledProcessError as e:
        return log_message(f"Error running commands in {directory}: {e}")

def process_file(file_path, config_path):
    # Extract the base name without extension
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Create new directory with the same name as the file
    new_dir = os.path.join('waveform_data', base_name)
    os.makedirs(new_dir, exist_ok=True)
    
    # Move the file to the new directory and rename it
    new_file_path = os.path.join(new_dir, 'output_file.mseed')
    shutil.move(file_path, new_file_path)
    
    # Copy config.yml to the new directory
    shutil.copy(config_path, new_dir)
    
    # Run seisglitch commands
    return run_seisglitch_commands(new_dir)

def main():
    # Check if config.yml exists
    config_path = 'config.yml'
    if not os.path.exists(config_path):
        with open('log.txt', 'a') as log_file:
            log_file.write(log_message("Error: config.yml not found in the current directory.\n"))
        return

    # Get all output_file{i}.mseed files in the waveform_data directory
    waveform_dir = 'waveform_data'
    files_to_process = [
        os.path.join(waveform_dir, filename)
        for filename in os.listdir(waveform_dir)
        if filename.startswith('output_file') and filename.endswith('.mseed')
    ]

    # Process files in parallel using ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=20) as executor:
        future_to_file = {executor.submit(process_file, file_path, config_path): file_path for file_path in files_to_process}
        for future in as_completed(future_to_file):
            file_path = future_to_file[future]
            try:
                result = future.result()
                with open('log.txt', 'a') as log_file:
                    log_file.write(f"{result}\n")
            except Exception as exc:
                with open('log.txt', 'a') as log_file:
                    log_file.write(log_message(f'{file_path} generated an exception: {exc}\n'))

if __name__ == "__main__":
    main()


Reading config file:
/home/abdallah/Documents/temp/nasa/waveform_data/output_file91/config.yml

Reading config file:
/home/abdallah/Documents/temp/nasa/waveform_data/output_file327/config.yml

  -----------------------
  RUNNING GLITCH DETECTOR
  -----------------------

  -----------------------
  RUNNING GLITCH DETECTOR
  -----------------------

Reading config file:
/home/abdallah/Documents/temp/nasa/waveform_data/output_file188/config.yml




Reading config file:
/home/abdallah/Documents/temp/nasa/waveform_data/output_file320/config.yml
  -----------------------
  RUNNING GLITCH DETECTOR
  -----------------------

INFO: Analysing seismic traces of file: 1/1
output_file.mseed

Reading config file:
/home/abdallah/Documents/temp/nasa/waveform_data/output_file313/config.yml

INFO: Analysing seismic traces of file: 1/1
  XB.ELYSE.02.BHU | 2019-09-28T05:21:55.074000Z - 2019-09-28T05:38:44.874000Z | 20.0 Hz, 20197 samplesoutput_file.mseed
  XB.ELYSE.02.BHU | 2021-06-29T10:00:52.405000Z -

Traceback (most recent call last):
  File "/home/abdallah/miniconda3/envs/seisglitch/bin/seisglitch", line 99, in <module>
    detect.detect( *waveform_files, inventory_file=inventory_file, **detect_params )
  File "/home/abdallah/miniconda3/envs/seisglitch/lib/python3.7/site-packages/seisglitch/detect.py", line 210, in detect
    stRAW.set_inventory(source=inventory_file)      # get & set station inventory for stream (response information)
  File "/home/abdallah/miniconda3/envs/seisglitch/lib/python3.7/site-packages/seisglitch/util.py", line 504, in set_inventory
    inventory      = self._get_inventory(source=source, verbose=verbose)
  File "/home/abdallah/miniconda3/envs/seisglitch/lib/python3.7/site-packages/seisglitch/util.py", line 493, in _get_inventory
    level     = 'response')                
  File "/home/abdallah/miniconda3/envs/seisglitch/lib/python3.7/site-packages/obspy/clients/fdsn/client.py", line 717, in get_stations
    raise ValueError(msg)
ValueError: The current 


Reading config file:
/home/abdallah/Documents/temp/nasa/waveform_data/output_file322/config.yml

  -----------------------
  RUNNING GLITCH DETECTOR
  -----------------------

Reading config file:
/home/abdallah/Documents/temp/nasa/waveform_data/output_file57/config.yml

  -----------------------
  RUNNING GLITCH DETECTOR
  -----------------------


INFO: Analysing seismic traces of file: 1/1
output_file.mseed
  XB.ELYSE.02.BHU | 2021-06-25T03:30:44.241000Z - 2021-06-25T03:45:58.041000Z | 20.0 Hz, 18277 samples
  XB.ELYSE.02.BHV | 2021-06-25T03:30:44.241000Z - 2021-06-25T03:45:58.041000Z | 20.0 Hz, 18277 samples
  XB.ELYSE.02.BHW | 2021-06-25T03:30:44.241000Z - 2021-06-25T03:45:58.041000Z | 20.0 Hz, 18277 samples

INFO: Preparing data.
  XB.ELYSE.02.BHU | 2021-06-25T03:30:44.241000Z - 2021-06-25T03:45:58.041000Z | 20.0 Hz, 18277 samples
  XB.ELYSE.02.BHV | 2021-06-25T03:30:44.241000Z - 2021-06-25T03:45:58.041000Z | 20.0 Hz, 18277 samples
  XB.ELYSE.02.BHW | 2021-06-25T03:30:44.241000Z

Traceback (most recent call last):
  File "/home/abdallah/miniconda3/envs/seisglitch/bin/seisglitch", line 99, in <module>
    detect.detect( *waveform_files, inventory_file=inventory_file, **detect_params )
  File "/home/abdallah/miniconda3/envs/seisglitch/lib/python3.7/site-packages/seisglitch/detect.py", line 210, in detect
    stRAW.set_inventory(source=inventory_file)      # get & set station inventory for stream (response information)
  File "/home/abdallah/miniconda3/envs/seisglitch/lib/python3.7/site-packages/seisglitch/util.py", line 504, in set_inventory
    inventory      = self._get_inventory(source=source, verbose=verbose)
  File "/home/abdallah/miniconda3/envs/seisglitch/lib/python3.7/site-packages/seisglitch/util.py", line 484, in _get_inventory
    client = Client(source)
  File "/home/abdallah/miniconda3/envs/seisglitch/lib/python3.7/site-packages/obspy/clients/fdsn/client.py", line 263, in __init__
    self._discover_services()
  File "/home/abdallah/miniconda3/envs/s


Reading config file:
/home/abdallah/Documents/temp/nasa/waveform_data/output_file355/config.yml

  -----------------------
  RUNNING GLITCH DETECTOR
  -----------------------

Reading config file:
/home/abdallah/Documents/temp/nasa/waveform_data/output_file325/config.yml


INFO: Analysing seismic traces of file: 1/1
output_file.mseed
  XB.ELYSE.02.BHU | 2021-06-30T06:50:56.834000Z - 2021-06-30T07:14:14.134000Z | 20.0 Hz, 27947 samples
  XB.ELYSE.02.BHV | 2021-06-30T06:50:56.834000Z - 2021-06-30T07:14:14.134000Z | 20.0 Hz, 27947 samples
  XB.ELYSE.02.BHW | 2021-06-30T06:50:56.834000Z - 2021-06-30T07:14:14.134000Z | 20.0 Hz, 27947 samples

INFO: Preparing data.
  XB.ELYSE.02.BHU | 2021-06-30T06:50:56.834000Z - 2021-06-30T07:14:14.134000Z | 20.0 Hz, 27947 samples
  XB.ELYSE.02.BHV | 2021-06-30T06:50:56.834000Z - 2021-06-30T07:14:14.134000Z | 20.0 Hz, 27947 samples
  XB.ELYSE.02.BHW | 2021-06-30T06:50:56.834000Z - 2021-06-30T07:14:14.134000Z | 20.0 Hz, 27947 samples

  -------------------

...

Glitch 000001,  2021-06-18T01:59:34,  V,  var_red=92.8 %.  Correction is done.  Acc. step =    4.1 nm/s**2
Glitch 000006,  2021-03-27T05:44:03,  U,  var_red=90.9 %.  Correction is done.  Acc. step =   17.5 nm/s**2
Glitch 000001,  2021-07-02T11:11:35,  V,  var_red=91.7 %.  Correction is done.  Acc. step =    5.0 nm/s**2.  Dis. step = -193.1 pm (var_red_spi= 9.4 %, Tgli-Tspi=-0.225 s)
Glitch 000002,  2020-01-26T12:18:02,  W,  var_red=34.8 %.  No correction done.
Glitch 000006,  2021-03-28T23:14:30,  U,  var_red=96.6 %.  Correction is done.  Acc. step =    8.9 nm/s**2.  Dis. step =  232.5 pm (var_red_spi= 8.0 %, Tgli-Tspi= 0.200 s)

OUTPUT GLITCH REMOVER:
#   +-----------------------------------------------------------------------------------+
#   | GLITCH DECTECTOR FILE(S):                                                         |
#   |   glitches_output_file.txt                                                        |
#   |                                                                

Traceback (most recent call last):
  File "/home/abdallah/miniconda3/envs/seisglitch/bin/seisglitch", line 108, in <module>
    remove.remove( *glitch_detector_files, waveform_files=waveform_files, inventory_file=inventory_file, **remove_params )
  File "/home/abdallah/miniconda3/envs/seisglitch/lib/python3.7/site-packages/seisglitch/remove.py", line 243, in remove
    glitches         = all_glitches[ (all_glitches[:,1]>=str(trace.stats.starttime)) & (all_glitches[:,2]<=str(trace.stats.endtime)) ]
IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed



  -----------------------
  RUNNING GLITCH DETECTOR
  -----------------------

  -----------------------
  RUNNING GLITCH DETECTOR
  -----------------------


INFO: Analysing seismic traces of file: 1/1
output_file.mseed
  XB.ELYSE.02.BHU | 2021-07-20T00:15:32.954000Z - 2021-07-20T00:37:28.854000Z | 20.0 Hz, 26319 samples
  XB.ELYSE.02.BHV | 2021-07-20T00:15:32.955000Z - 2021-07-20T00:37:28.855000Z | 20.0 Hz, 26319 samples
  XB.ELYSE.02.BHW | 2021-07-20T00:15:32.955000Z - 2021-07-20T00:37:28.855000Z | 20.0 Hz, 26319 samples
Glitch 000010,  2019-09-28T03:58:05,  U,  var_red=52.5 %.  No correction done.



INFO: Analysing seismic traces of file: 1/1
output_file.mseed
  XB.ELYSE.02.BHU | 2021-07-19T21:27:34.710000Z - 2021-07-19T22:07:02.510000Z | 20.0 Hz, 47357 samples
  XB.ELYSE.02.BHV | 2021-07-19T21:27:34.709000Z - 2021-07-19T22:07:02.509000Z | 20.0 Hz, 47357 samples
  XB.ELYSE.02.BHW | 2021-07-19T21:27:34.709000Z - 2021-07-19T22:07:02.509000Z | 20.0 Hz, 47357 samples
INFO: Preparing 

Traceback (most recent call last):
  File "/home/abdallah/miniconda3/envs/seisglitch/bin/seisglitch", line 108, in <module>
    remove.remove( *glitch_detector_files, waveform_files=waveform_files, inventory_file=inventory_file, **remove_params )
  File "/home/abdallah/miniconda3/envs/seisglitch/lib/python3.7/site-packages/seisglitch/remove.py", line 243, in remove
    glitches         = all_glitches[ (all_glitches[:,1]>=str(trace.stats.starttime)) & (all_glitches[:,2]<=str(trace.stats.endtime)) ]
IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed


Glitch 000007,  2019-06-03T03:53:44,  U,  var_red=98.4 %.  Correction is done.  Acc. step =  -16.4 nm/s**2.  Dis. step = -386.8 pm (var_red_spi=15.3 %, Tgli-Tspi=-0.375 s)
Glitch 000002,  2021-06-27T08:23:23,  V,  var_red=89.4 %.  Correction is done.  Acc. step =    8.5 nm/s**2.  Dis. step = -594.2 pm (var_red_spi=21.0 %, Tgli-Tspi= 0.025 s)
Glitch 000005,  2021-07-14T15:55:32,  U,  var_red=86.2 %.  Correction is done.  Acc. step =   -3.8 nm/s**2
Glitch 000001,  2021-07-02T12:09:34,  W,  var_red=95.5 %.  Correction is done.  Acc. step =   -3.7 nm/s**2.  Dis. step =  142.6 pm (var_red_spi=14.5 %, Tgli-Tspi= 0.000 s)
Glitch 000010,  2019-09-28T03:58:05,  V,  var_red=59.0 %.  No correction done.
Glitch 000010,  2021-07-20T00:34:52,  U,  var_red=67.2 %.  No correction done.
Glitch 000003,  2021-06-07T19:02:11,  V,  var_red=97.3 %.  Correction is done.  Acc. step =    6.7 nm/s**2.  Dis. step = -221.0 pm (var_red_spi=19.7 %, Tgli-Tspi= 0.025 s)
Glitch 000008,  2021-07-22T08:18:20,  V,  var_r